In [1]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import io
import imageio
from IPython.display import Image, display
from ipywidgets import widgets, Layout, HBox
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

import os
import glob
import numpy as np
import cv2

2023-04-28 11:07:22.686116: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-28 11:07:24.048175: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
## model load
model = keras.models.load_model('/home/siwon/dev/Deeplearning-6/model/gray_ball_model.h5')

2023-04-28 11:07:25.862180: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-28 11:07:25.967666: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-28 11:07:25.968002: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
base_path = "/home/siwon/dev/Deeplearning-6/data_video/video_frame_ball"

# 폴더 목록 가져오기
folder_list = sorted([d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))])

# 이미지를 읽는 함수
def read_image(file):
    return cv2.imread(file, cv2.IMREAD_GRAYSCALE)

# 각 폴더에서 이미지를 읽어 시퀀스로 저장
sequences = []
for folder in tqdm(folder_list, desc="Processing folders"):
    folder_path = os.path.join(base_path, folder)
    image_files = sorted(glob.glob(folder_path + "/*.jpg"))

    # 이미지를 그레이스케일로 읽고 numpy 배열로 변환
    with ThreadPoolExecutor() as executor:
        images = list(executor.map(read_image, image_files))
    
    sequence = np.array(images)
    sequences.append(sequence)

Processing folders: 100%|██████████| 100/100 [00:00<00:00, 609.96it/s]


In [4]:
# 시퀀스를 numpy 배열로 변환
dataset = np.array(sequences)

# 4차원으로 변환
dataset = np.expand_dims(dataset, axis=-1)

# Normalize pixel values to be between 0 and 1
dataset = dataset.astype("float32") / 255.0

In [5]:
# Select a few random examples from the dataset.
examples = dataset

# Iterate over the examples and predict the frames.
predicted_videos = []
for example in examples:
    # Pick the first/last ten frames from the example.
    frames = example[:10, ...]
    original_frames = example[10:, ...]
    new_predictions = np.zeros(shape=(10, *frames[0].shape))

    # Predict a new set of 10 frames.
    for i in range(10):
        # Extract the model's prediction and post-process it.
        frames = example[: 10 + i + 1, ...]
        new_prediction = model.predict(np.expand_dims(frames, axis=0))
        new_prediction = np.squeeze(new_prediction, axis=0)
        predicted_frame = np.expand_dims(new_prediction[-1, ...], axis=0)

        # Extend the set of prediction frames.
        new_predictions[i] = predicted_frame

    # Create and save GIFs for each of the ground truth/prediction images.
    for frame_set in [original_frames, new_predictions]:
        # Construct a GIF from the selected video frames.
        current_frames = np.squeeze(frame_set)
        current_frames = current_frames[..., np.newaxis] * np.ones(3)
        current_frames = (current_frames * 255).astype(np.uint8)
        current_frames = list(current_frames)

        # Construct a GIF from the frames.
        with io.BytesIO() as gif:
            imageio.mimsave(gif, current_frames, "GIF", fps=5)
            predicted_videos.append(gif.getvalue())

# Display the videos.
print(" Truth\tPrediction")
for i in range(0, len(predicted_videos), 2):
    # Construct and display an `HBox` with the ground truth and prediction.
    box = HBox(
        [
            widgets.Image(value=predicted_videos[i]),
            widgets.Image(value=predicted_videos[i + 1]),
        ]
    )
    display(box)

2023-04-28 11:07:29.570208: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8801


1/1 [==============================] - 0s 38ms/step
 Truth	Prediction


In [7]:
import os

save_dir = "/home/siwon/dev/Deeplearning-6/data_video/gif"
os.makedirs(save_dir, exist_ok=True)
# Display the videos.


for i in range(0, len(predicted_videos), 2):
    # Save the ground truth video
    with open(os.path.join(save_dir, f"truth_video_{i // 2}.gif"), "wb") as f:
        f.write(predicted_videos[i])

    # Save the predicted video
    with open(os.path.join(save_dir, f"predicted_video_{i // 2}.gif"), "wb") as f:
        f.write(predicted_videos[i + 1])
